want to be able to easily join all datasets on id, so have all info per sample_id

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000

### read in metadata

In [3]:
meta_df = pd.read_csv('../VioAndMetadata_Cleaning/sampleFromQiitaDownload.txt',sep='\t', low_memory=False)

In [4]:
meta_df.shape

(25830, 594)

In [5]:
#meta_df.columns

In [6]:
meta_df[['sample_name', 'survey_id', 'qiita_study_id']].head(2)

,sample_name,survey_id,qiita_study_id
0,10317.000001000,48c83b76031b7580,10317
1,10317.000001001,0b0832aafb76068d,10317


In [7]:
meta_df[['sample_name', 'survey_id']].sort_values('sample_name').head(2)

,sample_name,survey_id
0,10317.000001000,48c83b76031b7580
1,10317.000001001,0b0832aafb76068d


In [8]:
meta_df[meta_df['sample_name'].str.contains('Blank')].shape

(176, 594)

### read in drug data dense

In [9]:
drug_df = pd.read_csv('../drug_data/drugbank_drug_data.csv')
print(drug_df.shape)

(53271, 11)


In [10]:
drug_df.head(2)

,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,0,72291,0000904d9779a86c,MEDICATION_LIST,"Pravastatin, CoQ10",2,C10AA03,C,Cardiovascular system,Pravastatin,False
1,1,12914,000a1da7d9d7e35b,ANTIBIOTIC_MED,Zymar (gatifloxacin) eye drop (NOT ORAL),1,J01MA16,J,Antiinfectives for systemic use,Gatifloxacin,NaN


In [11]:
drug_df.sort_values('sample_name').head(2)

,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
12666,12666,1000,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3,N06AB10,N,Nervous system,Escitalopram,False
12668,12668,1000,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3,N06AX12,N,Nervous system,Bupropion,False


### create dataframes with joinable ids

In [12]:
def meta_df_id_clean(meta_df):
    meta_df['sample_id'] = meta_df['sample_name'].apply(lambda x: x.split('.')[1]) 
    meta_df['sample_id'] = pd.to_numeric(meta_df['sample_id'], errors='coerce', downcast='integer')
    meta_df_clean = meta_df.dropna(subset=['sample_id']).reset_index()
    meta_df_clean['sample_id'] = meta_df_clean['sample_id'].apply(lambda x: int(x))
    return meta_df_clean

In [13]:
def drug_df_id_clean(drug_df):
    drug_df['sample_id'] = drug_df['sample_name'].apply(lambda x: str(x).split('.')[0])
    drug_df['sample_id'] = drug_df['sample_id'].astype('int')
    return drug_df

clean metadata sample_id that will match with drug_id

In [14]:
meta_clean_id_df = meta_df_id_clean(meta_df)

In [15]:
print(meta_clean_id_df.shape)
meta_clean_id_df[['sample_id', 'sample_name']].head(2)

(22644, 596)


,sample_id,sample_name
0,1000,10317.000001000
1,1001,10317.000001001


clean drug_df

In [16]:
drug_clean_id_df = drug_df_id_clean(drug_df)

In [17]:
print(drug_clean_id_df.shape)
drug_clean_id_df.head(2)

(53271, 12)


,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal,sample_id
0,0,72291,0000904d9779a86c,MEDICATION_LIST,"Pravastatin, CoQ10",2,C10AA03,C,Cardiovascular system,Pravastatin,False,72291
1,1,12914,000a1da7d9d7e35b,ANTIBIOTIC_MED,Zymar (gatifloxacin) eye drop (NOT ORAL),1,J01MA16,J,Antiinfectives for systemic use,Gatifloxacin,NaN,12914


try joing drug_data and metadata on id

In [18]:
merge_df = pd.merge(meta_clean_id_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')

In [19]:
merge_df.shape

(60948, 13)

In [20]:
merge_df.head()

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True
2,10317.000001000,1000,12668.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AX12,N,Nervous system,Bupropion,False
3,10317.000001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10317.000001002,1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


lets split vioscreen and metadata columns

In [21]:
col_names = pd.Series(meta_clean_id_df.columns)
meta_col_names = list(col_names[~col_names.str.contains('vioscreen')])
vio_col_names = list(col_names[col_names.str.contains('vioscreen')])
vio_col_names = vio_col_names + ['sample_name', 'sample_id', 'survey_id']

In [22]:
print(len(meta_col_names))
print(len(vio_col_names))

323
276


In [23]:
meta_agp_df = meta_clean_id_df[meta_col_names]
print(meta_agp_df.shape)
meta_agp_df[['sample_id']].head(2)

(22644, 323)


,sample_id
0,1000
1,1001


In [24]:
vio_df = meta_clean_id_df[vio_col_names]
print(vio_df.shape)
vio_df.head(2)

(22644, 276)


,vioscreen_a_bev,vioscreen_a_cal,vioscreen_acesupot,vioscreen_activity_level,vioscreen_add_sug,vioscreen_addsugar,vioscreen_adsugtot,vioscreen_age,vioscreen_alanine,vioscreen_alcohol,vioscreen_alcohol_servings,vioscreen_alphacar,vioscreen_alphtoce,vioscreen_alphtoco,vioscreen_arginine,vioscreen_ash,vioscreen_aspartam,vioscreen_aspartic,vioscreen_avcarb,vioscreen_bcodeid,vioscreen_betacar,vioscreen_betacryp,vioscreen_betaine,vioscreen_betatoco,vioscreen_biochana,vioscreen_bmi,vioscreen_caffeine,vioscreen_calcium,vioscreen_calcium_avg,vioscreen_calcium_dose,vioscreen_calcium_freq,vioscreen_calcium_from_dairy_servings,vioscreen_calcium_servings,vioscreen_calories,vioscreen_carbo,vioscreen_cholest,vioscreen_choline,vioscreen_clac9t11,vioscreen_clat10c12,vioscreen_copper,vioscreen_coumest,vioscreen_cystine,vioscreen_d_cheese,vioscreen_d_milk,vioscreen_d_tot_soym,vioscreen_d_total,vioscreen_d_yogurt,vioscreen_daidzein,vioscreen_database,vioscreen_delttoco,vioscreen_discfat_oil,vioscreen_discfat_sol,vioscreen_dob,vioscreen_eer,vioscreen_email,vioscreen_erythr,vioscreen_f_citmlb,vioscreen_f_nj_citmlb,vioscreen_f_nj_other,vioscreen_f_nj_total,vioscreen_f_other,vioscreen_f_total,vioscreen_fat,vioscreen_fiber,vioscreen_fibh2o,vioscreen_fibinso,vioscreen_finished,vioscreen_fish_servings,vioscreen_fol_deqv,vioscreen_fol_nat,vioscreen_fol_syn,vioscreen_formontn,vioscreen_fried_fish_servings,vioscreen_fried_food_servings,vioscreen_frt5_day,vioscreen_frtsumm,vioscreen_fructose,vioscreen_fruit_servings,vioscreen_g_nwhl,vioscreen_g_total,vioscreen_g_whl,vioscreen_galactos,vioscreen_gammtoco,vioscreen_gender,vioscreen_genistn,vioscreen_glac,vioscreen_gltc,vioscreen_glucose,vioscreen_glutamic,vioscreen_glycine,vioscreen_glycitn,vioscreen_grams,vioscreen_hei2010_dairy,vioscreen_hei2010_empty_calories,vioscreen_hei2010_fatty_acids,vioscreen_hei2010_fruit,vioscreen_hei2010_greens_beans,vioscreen_hei2010_protien_foods,vioscreen_hei2010_refined_grains,vioscreen_hei2010_score,vioscreen_hei2010_sea_foods_plant_protiens,vioscreen_hei2010_sodium,vioscreen_hei2010_veg,vioscreen_hei2010_whole_fruit,vioscreen_hei2010_whole_grains,vioscreen_hei_drk_g_org_veg_leg,vioscreen_hei_fruit,vioscreen_hei_grains,vioscreen_hei_meat_beans,vioscreen_hei_milk,vioscreen_hei_non_juice_frt,vioscreen_hei_oils,vioscreen_hei_sat_fat,vioscreen_hei_score,vioscreen_hei_sodium,vioscreen_hei_sol_fat_alc_add_sug,vioscreen_hei_veg,vioscreen_hei_whl_grains,vioscreen_height,vioscreen_histidin,vioscreen_inositol,vioscreen_iron,vioscreen_isoleuc,vioscreen_isomalt,vioscreen_joules,vioscreen_juice_servings,vioscreen_lactitol,vioscreen_lactose,vioscreen_legumes,vioscreen_leucine,vioscreen_line_gi,vioscreen_low_fat_dairy_serving,vioscreen_lutzeax,vioscreen_lycopene,vioscreen_lysine,vioscreen_m_egg,vioscreen_m_fish_hi,vioscreen_m_fish_lo,vioscreen_m_frank,vioscreen_m_meat,vioscreen_m_mpf,vioscreen_m_nutsd,vioscreen_m_organ,vioscreen_m_poult,vioscreen_m_soy,vioscreen_magnes,vioscreen_maltitol,vioscreen_maltose,vioscreen_mangan,vioscreen_mannitol,vioscreen_methhis3,vioscreen_methion,vioscreen_mfa141,vioscreen_mfa161,vioscreen_mfa181,vioscreen_mfa201,vioscreen_mfa221,vioscreen_mfatot,vioscreen_multi_calcium_avg,vioscreen_multi_calcium_dose,vioscreen_multivitamin,vioscreen_multivitamin_freq,vioscreen_natoco,vioscreen_nccglbr,vioscreen_nccglgr,vioscreen_niacin,vioscreen_niacineq,vioscreen_nitrogen,vioscreen_non_fried_fish_servings,vioscreen_nutrient_recommendation,vioscreen_omega3,vioscreen_oxalic,vioscreen_oxalicm,vioscreen_pantothe,vioscreen_pectins,vioscreen_pfa182,vioscreen_pfa183,vioscreen_pfa184,vioscreen_pfa204,vioscreen_pfa205,vioscreen_pfa225,vioscreen_pfa226,vioscreen_pfatot,vioscreen_phenylal,vioscreen_phosphor,vioscreen_phytic,vioscreen_pinitol,vioscreen_potass,vioscreen_procdate,vioscreen_proline,vioscreen_protanim,vioscreen_protein,vioscreen_protocol,vioscreen_protveg,vioscreen_questionnaire,vioscreen_recno,vioscreen_retinol,vioscreen_rgrain,vioscreen_ribofla,vioscreen_sacchar,vioscree

test merges to make sure works before exporting datasets to csv

In [25]:
merge_df2 = pd.merge(meta_agp_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')
merge_df2.shape
merge_df2.head(2)

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True


In [26]:
merge_df3 = pd.merge(vio_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')
merge_df3.shape
merge_df3.head(2)

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True


export to .csv

In [26]:
#meta_agp_df.to_csv('agp_only_meta.csv', index=False)

In [27]:
#vio_df.to_csv('vio_only.csv', index=False)

export versions of drug data to .csv

In [28]:
drug_df = pd.read_csv('../drug_data/drugbank_drug_data.csv')
drug_clean_id_df = drug_df_id_clean(drug_df)
print(drug_clean_id_df.shape)

(53271, 12)


In [29]:
drug_clean_id_df[['sample_id']].head(2)

,sample_id
0,72291
1,12914


In [30]:
drug_clean_id_df.to_csv('v2_drugbank_drug_data.csv', index=False)

In [31]:
drug_df = pd.read_csv('../drug_data/drug_data_dense.csv')
drug_clean_id_df = drug_df_id_clean(drug_df)
print(drug_clean_id_df.shape)

(23491, 7)


In [32]:
drug_clean_id_df[['sample_id']].head(2)

,sample_id
0,72291
1,43062


In [33]:
#drug_clean_id_df.to_csv('v2_drug_data_dense.csv', index=False)

## biom table data join

In [34]:
biom_df = pd.read_pickle('../AGP/biom_data_v3.pkl')

In [35]:
print(type(biom_df))
biom_df.shape

<class 'pandas.core.sparse.frame.SparseDataFrame'>


(14657, 376924)

In [36]:
biom_df.head()

,TACGAAGGGGGCTAGCGTTGCTCGGAATCACCGGGCGTAAAGGGCGCGTAGGCGGACTCTTAAGTCGGGGGTGAAAGCCCAGGGCTCAACCCTGGAATTG,TACGTAGGGAGCGAGCGTTGTCCGGATTTACTGGGTGTAAAGGGTGCGTAGGCGGCCGAGCAAGTCAGTTGTGAAAACTCTGGGCTTAACCCATAACGTG,TACGGAGGGAGCTAGCGTTGTTCGGATATACTGGGCGTAAAGCGCACGTAGGCGGCGTCGTAAGTCAGGGGTGAAATCCCGGAGCTCAACTCCGGAACTG,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTGAGTAGGCGGCACGGCAAGTAAGATGTGAAAGCCCGAGGCTTAACCTCGGGATTG,TACGGGGGATCCAAGCGTTATCCGGAATCATTGGGTTTAAAGGGTCCGTAGGTGGATGATTAAGTCAGAGGTGAAATCCTGCAGCTCAACTGTAGAATTG,AACGTAGGGGGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGAGCGTAGGCGGCCAATTAAGTCAGATGTGAAATTCCCGGGCTTAACCTGGGCAGTG,TACGGAGGGTGCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTCCGTAGGCTGATTTGTAAGTCAGTGGTGAGATCTCACAGCTCAACTGTGAAACTG,TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCACGCAGGCGGTTAGTTAAGTCAGATGTGATAACTATGGGCTCAACCCATAGCCTG,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAACGTAGACGGCATGGCAAGCCAGATGTGAAAGCCCGGGGCTCAACCCCGTGACTG,TACGTAGGGAGCGAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGTAGACGGATGGACAAGTCTGATGTGAAGGGCTGGGGCTCAACCCCGGGACTG,AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGTAGGCGGTCTGACAAGTCAGAAGTGAAAGCCCGGGGCTCAACCCCGGGACTG,TACGTAGGTGGCGAGCGTTATCCGGAATTACTGGGCGTAAAGGGCGCGTAGGCGGCCAATTAAGTTAGATGTGAAATCCCCGGGCTTAACCTGGGTGTTG,TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCATAAAGCGTGCGTAGACGGTTTATTAAGTCCAGAATTAAAGCCCGAGGCTTAACCTCGGTTCGT,TACGTAGGGGGCAAGCGTTGTCCGGATTTACTGGGTGTAAAGGGCGCGTAGGCGGGCTCGTAAGTCAGAGGTGAAATCCTGCAGCTTAACTGCAGAACTG,AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGCGTAAAGCGCACGCGGGCGGTTTGTTAAGTCAGATGTGAAATCCCCGAGCTCAACCTGGGAACTG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACCATCAAGTCAGCGGTCAAATGTCGGGGCTCAACCCCGTAAAGC,TACGTAGGTGGCGAGCGTTATCCGGATTTATTGGGCGTAAAGGGTGCGCAGACGGTTTATTAAGCCTAAAATCAAATCTTGGGGCTTAACCCCATTCCGT,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGATGGGCAAGTCTGATGTGAAAGTTTGCGGCTCAACCGTAAAATTG,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCATAAAGGGCATCTAGGCGGCCTTTCAAGTCAGGGGTGAAAGCCTGCGGCTCAACCGCAGGCCTG,AACGTAGGTCACAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTTCTTAAGTCTGATGTGAAAGCCCCCGGTTCAACCGGGGAGGGT,TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGGGCGCAGGCGGTTCCTTAAGTCTGATGTGAAAGCCCCCGGCTCAACCGGGGAGGGT,TACGTAGGTGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGGAAAGCAAGTCAGCTGTGAAAACTACAGGCTTAACCTGTAGACTG,TACGTATGGGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGTGCGTAGGTGGTGGTTTAAGCGCAGGGTTTAAGGCAATGGCTTAACTATTGTTCGC,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGAAGGGCAAGTCTGGAGTGAAAGCCCGGGGCTCAACCCCGGGACTG,TACGTATGGTGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGCGAGATAAGTCTGAGGTAAAAGCCCGTGGCTCAACCACGGTAAGC,TACGTAGGGAGCGAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGTAGGCGGGATGACAAGTAGAATGTGAAAACAGCCGGCTCAACTGGTTGTTGC,TACGTAGGTGGCAAGCGTTGTCCTGATTTACTGGGTGTAAAGGGCGTGTAGGCGGAGCAGCAAGTCAGAAGTGAAATCTCTGGGCTCAACCCAGAAACTG,CACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTGTGTCGCGTCGGCCGTGAAACTTGGGGCTTAACTCTGAGCGTGC,GACGTAGGGCGCGAGCGTTGTCCGGATTTATTGGGCGTAAAGAGCTCGTAGGCGGCTTGTCGCGTCGTCTGTGAAAACCCGTGGCTCAACTGCGGGCTTG,GACGGAGGGTGCAAACGTTGTTCGGAATCACTGGGCATAAAGAGCATGTAGGCTGTCTAGTCAGTCGGATGTGAAAGCCCGCGGCTCAACCGCGGAACTG,TACGAAGGGGGCAAGCGTTGTTCGGGATTACTGGGCGTAAAGGGCGCGTAGGCGGCCATCTTTGTCAGGCGTGAAAGCCCTGGTCTCAACCCAGGAATTG,TACGGAAGGTCCGGGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCCGGAGATTGAGCGTGTTGTGAAATGTAGATGCTCAACATCTGCACTG,CACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGACGGCTGTGCAAGTCTGAAGTGAAAGGCATGGGCTCAACCTGTGGACTG,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGGATCGCAAGTCAGATGTGAAAACTATGGGCTTAACCTCCAAACTG,TACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGTGTAGAGGGCGTGTAGCCGGGAAGGCAAGTCAGCGGTGAAATTTCCGGGCTCAACCCGGACTGTG,TACGTAGGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTGTGTAGGCGGGATTGCAAGTCAGATGTGAAAACTATGGGCTTAACCCATAAACTG,TACGTAGGGAGCAAGCGTTATCCGGATTTATTGGGTGTAAAGGGTGCGTAGACGGTAATGCAAGTTAGTTGTGAAATCCCTCGGCTTAACCGAAGAACTG,TACGTAGGGGGCGAGCGTTATCCGGATTCATTGGGCGTAAAGCGCGCGCAGGCGGATTGGTCAGTCTGTCTTAAAAGTGCGGGGCTTAACCCCGTGATGG,TACGTAGGGAGCAAGCGTTATCCGGATTTATTGGGCGTAAAGGGCGCGTAGGCGGCCTGCTAAGTCTGAAGTGAAAGTCCTGCTTTCAAGGTGGGAAGTG,TACGTAGGTCGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGAGCGCGCAGGCGGCTGGC

In [52]:
biom_df['sample_id'] = biom_df['sample_name'].apply(lambda x: x.split('.')[1]) 

In [53]:
biom_df['sample_id'] = pd.to_numeric(biom_df['sample_id'], errors='coerce', downcast='integer')

In [54]:
biom_join_df = biom_df.dropna(subset=['sample_id'])

In [55]:
biom_join_df['sample_id'] = biom_join_df['sample_id'].apply(lambda x: int(x))

//anaconda/envs/qiime2-2018.11/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
print(biom_join_df.shape)

(14624, 376925)


#### test merge between biom data and metadata

In [60]:
merge_df4 = pd.merge(meta_agp_df[['sample_name', 'sample_id']], 
                     biom_join_df[['sample_name', 'sample_id']], how='right', on='sample_id')
print(merge_df4.shape)
merge_df4.head(2)

(14624, 3)


,sample_name_x,sample_id,sample_name_y
0,10317.000001000,1000,10317.000001000.33048
1,10317.000001001,1001,10317.000001001.27725


In [61]:
merge_df4.isna().sum()

sample_name_x    0
sample_id        0
sample_name_y    0
dtype: int64

#### looking good!

In [62]:
biom_join_df.to_pickle('id_clean_biom.pkl')

### Messing around with metadata

In [47]:
meta_agp_df[meta_agp_df['sample_id'].isin(alist)].env_material.value_counts()

feces     11982
saliva      792
sebum       771
Name: env_material, dtype: int64

In [48]:
meta_agp_df.salted_snacks_frequency.value_counts()

Rarely (less than once/week)     6266
Not provided                     5304
Occasionally (1-2 times/week)    4525
Never                            2860
Regularly (3-5 times/week)       2642
Daily                             733
Unspecified                       230
not applicable                     10
Not applicable                      1
Name: salted_snacks_frequency, dtype: int64

In [49]:
meta_agp_df.env_material.value_counts()

feces           19407
saliva           1437
sebum            1308
mucus             319
ear wax            56
tears              54
Unspecified        44
food product       10
Name: env_material, dtype: int64

In [50]:
meta_agp_df.age_cat.unique()

array(['60s', '50s', '40s', '20s', '30s', '70+', 'Not provided', 'child',
       'teen', nan, 'baby', 'Unspecified', 'not applicable',
       'Not applicable'], dtype=object)